In [1]:
!ls


CONTRIBUTORS.md  download_model.py  requirements.txt  Untitled.ipynb
DEVELOPERS.md	 encode.py	    src
Dockerfile.cpu	 LICENSE	    train-horovod.py
Dockerfile.gpu	 README.md	    train.py


In [2]:
!pip3 install -r requirements.txt #

     |████████████████████████████████| 81kB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 604kB 27.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 17.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 13.9MB/s eta 0:00:01
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103543 sha256=38aafb71196bdc4923a7a62f1f448ba47f4bf3ada15d0658452aa7d306883f41
  Stored in directory: /home/ec2-user/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=564360 sha256=a9b431b3b24c4ad99f0961522a26cb60d3a87cb154d6a6abd6d2efa27c367c30
  Stored in directory: /home/ec2-user/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: requests 2.20.0
    Uninstalling requests-2.20.0:
      Successfully uninstalled requests-2.20.0
You should consider upgrading v

In [3]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.10Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 44.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.58Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:18, 75.4Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 12.8Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 42.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 35.6Mit/s]                                                       


In [5]:
!export PYTHONIOENCODING=UTF-8

In [9]:
import sys
sys.path.append("/home/ec2-user/SageMaker/gpt-2/src")

In [8]:
!pwd

/home/ec2-user/SageMaker/gpt-2


In [10]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

In [11]:
import interactive_conditional_samples

Fetching checkpoint: 1.00kit [00:00, 1.07Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 48.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.33Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:17, 79.7Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 9.70Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 49.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 37.3Mit/s]                                                       


In [12]:
#I copied the class out of src to here to modify it with visible ease, it can go back
class GPT2:


  # extracted from the source code to generate some text based on a prior
    def __init__(
        self,
        model_name='345M',
        seed=None,
        nsamples=1,
        batch_size=1,
        length=None,
        temperature=1,
        top_k=0,
        raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
        if batch_size is None:
            batch_size = 1
        assert nsamples % batch_size == 0

        self.nsamples = nsamples
        self.batch_size = batch_size

        self.enc = encoder.get_encoder(model_name)
        hparams = model.default_hparams()
        with open(os.path.join('models', model_name, 'hparams.json')) as f:
            hparams.override_from_dict(json.load(f))

        if length is None:
            length = hparams.n_ctx // 2
        elif length > hparams.n_ctx:
            raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

        self.sess = tf.Session(graph=tf.Graph())
        self.sess.__enter__()

        self.context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        self.ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(self.sess, self.ckpt)

    def close(self):
        self.sess.close()
  
    def generate_conditional(self,raw_text):
        context_tokens = self.enc.encode(raw_text)
        generated = 0
        for _ in range(self.nsamples // self.batch_size):
            out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
            })[:, len(context_tokens):]
            for i in range(self.batch_size):
                generated += 1
                text = self.enc.decode(out[i])
                return text
              #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
              #print(text)
        #print("=" * 80)


In [13]:
gpt2 = GPT2()





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


In [ ]:
#gpt2.close() #remember to close session after

In [15]:
# result = gpt2.generate_conditional(raw_text="Hello bot, tell me about yourself")

# print(result)

."

You ask if it is safe to question the codeword cherishing inside him.

"What gives girl you're here? Meet my old friend Krumkov? Someone from your past? Cause he screws the wrong people. Tell him, he should reach the Hotel talli Ive no doubt about it, it's nothing to kill for."

Penitent looks up at you, wide-eyed. "Galp..."

Search Warrant Pay

Darius Kurmov:

An Archaeologist and Sydney Esperang

There is major public activity on Piltover 27 in the form of an intense wave of actions. The simultaneous hijacking of all cargo ships, smuggling and reconnaissance operations linking Piltover with Germany and Tehran, is temporary and ineffectual. Fighters from Kazortas each declared for surgical mutiny, while engineers charged with bringing care into operation appear as helpless victims of pireface without fingers and who roam through vivid oxymorons left out of dinner talk. The City's AI is activated, except by Crossing Over. Inquirer Nolf refuses to utter the words of unimaginable gri

In [25]:
class Who:
    """A class defining the conversation parties: me, friend"""
    def __init__(self):
        self.prefixes = []

    def matches(self,phrase):
        for prefix in self.prefixes:
            if phrase.startswith(prefix):
            #print(f"{phrase} starts with {prefix}")
                return True

    #print(f"{phrase} does not start with {self.prefixes}")
        return False

    def get_random_prefix(self):
        return self.prefixes[0] #just use first one

class Friend(Who):
    def __init__(self):
        super().__init__()  #initializes prefixes from who
        self.prefixes = ["Friend said: \""]
   
  
class You(Who):
    def __init__(self):
        super().__init__() 
        self.prefixes = ["You said: \""]

In [33]:
class Chatter:
  
    def __init__(self, prior = None):
        if prior is None:
            prior="""
              You: "Hello, are you my friend?"
              Friend: "Of course!"
              """
        self.suggestion = None

        self.friend = Friend()
        self.you = You()
        self.parties  = [ self.friend, self.you ]

        self.conversation = []

        lines = prior.split("\n")
        for line in lines:
            line = line.strip()
            if len(line)!=0:
                party = None
                for party in self.parties:
                    if party.matches(line):
                        break
                if party is None:
                    raise Exception(f"Unknown party: {line}")
                self.conversation.append((party,line))
        self.get_suggestion()


    def get_prior(self):
        conv = ""
        for (party, line) in self.conversation:
            conv+=line+"\n"
        return conv

    def get_suggestion(self):
        who, last_line = self.conversation[-1]

        party_index = self.parties.index(who)
        next_party = self.parties[(party_index+1) % len(self.parties)]

        conv = self.get_prior()
        conv += next_party.get_random_prefix()
        answer = self.get_answer(next_party, conv)

        if not next_party.matches(answer):
            prefix = next_party.get_random_prefix()
            answer = prefix + answer

        self.suggestion = (next_party, answer)

    def next(self, party = None, answer = ""):
        """Continue the conversation
        :param party: None -> use the current party which is currently in turn
        :param answer: None -> use the suggestion, specify a text to override the 
               suggestion

        """
        suggested_party, suggested_answer = self.suggestion
        if party is None:
            party = suggested_party

        if answer == "":
            answer = suggested_answer

        if not party.matches(answer):
            prefix = party.get_random_prefix()
            answer = prefix + answer

        answer = answer.strip()
        if answer[-1] != "\"":
          # add the closing "
            answer += "\""

        self.conversation.append((party, answer))    
        self.get_suggestion()

    def retry(self):
        self.get_suggestion()

    def get_answer(self, party, conv):
        answer = gpt2.generate_conditional(raw_text=conv)
        lines = answer.split("\n")
        line = ""
        for line in lines:
            if line !="":
                break


        if line!="":
            return line

        return ""

    def show(self):
        conv = ""
        for (party, line) in self.conversation:
            conv+=line+"\n"
        print(conv)
        if self.suggestion is not None:
            party, answer  = self.suggestion
            print("--> "+answer)


In [34]:
c = Chatter()

In [35]:
c.show()

You: "Hello, are you my friend?"
Friend: "Of course!"

--> Friend said: "Please trust me."


In [36]:
c.retry()


In [37]:
c.next()

In [38]:
c.next(c.you, "My favorite video game is Overwatch")

In [39]:
c.show()

You: "Hello, are you my friend?"
Friend: "Of course!"
Friend said: "Let's head back to the train station."
You said: "My favorite video game is Overwatch"

--> Friend said: "Oh, not you."


In [40]:
c.retry()

In [41]:
c.next(c.you, "what is your favorite game?")
c.show()

You: "Hello, are you my friend?"
Friend: "Of course!"
Friend said: "Let's head back to the train station."
You said: "My favorite video game is Overwatch"
You said: "what is your favorite game?"

--> Friend said: "rolling dice."


In [42]:
c.next(c.you, "why do you like rolling dice?")

In [ ]:
c.show()